## 1. Cargar credenciales y ambiente

In [ ]:
!pip install -U ibm-watson-machine-learning

### Obtención de token

#### Si ejecutamos notebook desde CPD

In [9]:
import os
token = os.environ['USER_ACCESS_TOKEN']
wml_credentials = {
"token": token,
"instance_id" : "openshift",
"url": "https://cpd-cpd.apps.cp4d-education-install.cp.fyre.ibm.com/",
"version": "4.8"
}

#### Si ejecutamos fuera de CP4D

In [5]:
username = 'PASTE YOUR USERNAME HERE'
api_key = 'PASTE YOUR API_KEY HERE'
wml_credentials = {
"username" : username,
"apikey" : api_key,
"instance_id" : "openshift",
"url": "https://cpd-cpd.apps.cp4d-education-install.cp.fyre.ibm.com/",
"version": "4.8"
}

In [10]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

### Creamos un space

In [5]:
metadata = {
    client.spaces.ConfigurationMetaNames.NAME: "Fyre_space",
    client.spaces.ConfigurationMetaNames.DESCRIPTION: "Espacio creado con notebook de github",
    client.spaces.ConfigurationMetaNames.STAGE: {"production": True,
                                                 "name": "stage_name"},
    client.spaces.ConfigurationMetaNames.TYPE: "cpd",
}
spaces_details = client.spaces.store(meta_props=metadata)

Space has been created. However some background setup activities might still be on-going. Check for 'status' field in the response. It has to show 'active' before space can be used. If it's not 'active', you can monitor the state with a call to spaces.get_details(space_id). Alternatively, use background_mode=False when calling client.spaces.store().


In [11]:
space_id=client.spaces.list(limit=50)[0:1].ID[0]
print(space_id)

------------------------------------  ----------  ------------------------
ID                                    NAME        CREATED
333a62f2-0a84-45b9-b841-d8ebc8cf4c66  Fyre_space  2025-01-27T19:40:41.550Z
------------------------------------  ----------  ------------------------
333a62f2-0a84-45b9-b841-d8ebc8cf4c66


In [12]:
client.set.default_space(space_id)

'SUCCESS'

## 2. Cargar data y crear modelo

In [13]:
import sklearn
from sklearn import datasets

digits = datasets.load_digits()

In [14]:
samples_count = len(digits.images)
print("Number of samples: " + str(samples_count))
train_data = digits.data[: int(0.7*samples_count)]
train_labels = digits.target[: int(0.7*samples_count)]

test_data = digits.data[int(0.7*samples_count): int(0.9*samples_count)]
test_labels = digits.target[int(0.7*samples_count): int(0.9*samples_count)]

score_data = digits.data[int(0.9*samples_count): ]

print("Number of training records: " + str(len(train_data)))
print("Number of testing records : " + str(len(test_data)))
print("Number of scoring records : " + str(len(score_data)))

Number of samples: 1797
Number of training records: 1257
Number of testing records : 360
Number of scoring records : 180


In [15]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import svm, metrics

In [16]:
scaler = preprocessing.StandardScaler()

In [17]:
clf = svm.SVC(kernel='rbf')

In [18]:
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])

In [19]:
model = pipeline.fit(train_data, train_labels)


In [20]:
predicted = model.predict(test_data)

print("Evaluation report: \n\n%s" % metrics.classification_report(test_labels, predicted))

Evaluation report: 

              precision    recall  f1-score   support

           0       1.00      0.97      0.99        37
           1       0.97      0.97      0.97        34
           2       1.00      0.97      0.99        36
           3       1.00      0.94      0.97        35
           4       0.78      0.97      0.87        37
           5       0.97      0.97      0.97        38
           6       0.97      0.86      0.91        36
           7       0.92      0.97      0.94        35
           8       0.91      0.89      0.90        35
           9       0.97      0.92      0.94        37

    accuracy                           0.94       360
   macro avg       0.95      0.94      0.95       360
weighted avg       0.95      0.94      0.95       360



In [37]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-23.1-py3.10")

In [48]:
client.software_specifications.list()

------------------------------  ------------------------------------  ----  ------------  ------------------------------
NAME                            ID                                    TYPE  STATE         REPLACEMENT
default_py3.6                   0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  retired       runtime-22.2-py3.10
runtime-22.1-r3.6               018ebea5-1d1f-5fec-b93e-5e2ab30e7f38  base  retired       runtime-22.2-r4.2
autoai-ts_rt23.1-py3.10         01ce9391-1a79-5a33-94fb-2e134337f314  base  supported
kernel-spark3.2-scala2.12       020d69ce-7ac1-5e68-ac1a-31189867356a  base  retired
tensorflow_rt23.1-py3.10-edt    067048c7-c771-5933-9aa0-4f9fbdc92da8  base  supported
pytorch-onnx_1.3-py3.7-edt      069ea134-3346-5748-b513-49120e15d288  base  retired       pytorch-onnx_rt22.2-py3.10-edt
tensorflow_rt23.1-py3.10        079a91e0-245f-5269-8926-3c20b28f37dc  base  supported
scikit-learn_0.20-py3.6         09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  retired       runtim

,NAME,ID,TYPE,STATE,REPLACEMENT
0,default_py3.6,0062b8c9-8b7d-44a0-a9b9-46c416adcbd9,base,retired,runtime-22.2-py3.10
1,runtime-22.1-r3.6,018ebea5-1d1f-5fec-b93e-5e2ab30e7f38,base,retired,runtime-22.2-r4.2
2,autoai-ts_rt23.1-py3.10,01ce9391-1a79-5a33-94fb-2e134337f314,base,supported,
3,kernel-spark3.2-scala2.12,020d69ce-7ac1-5e68-ac1a-31189867356a,base,retired,
4,tensorflow_rt23.1-py3.10-edt,067048c7-c771-5933-9aa0-4f9fbdc92da8,base,supported,
5,pytorch-onnx_1.3-py3.7-edt,069ea134-3346-5748-b513-49120e15d288,base,retired,pytorch-onnx_rt22.2-py3.10-edt
6,tensorflow_rt23.1-py3.10,079a91e0-245f-5269-8926-3c20b28f37dc,base,supported,
7,scikit-learn_0.20-py3.6,09c5a1d0-9c1e-4473-a344-eb7b665ff687,base,retired,runtime-22.2-py3.10
8,spark-mllib_3.0-scala_2.12,09f4cff0-90a7-5899-b9ed-1ef348aebdee,base,retired,
9,pytorch-onnx_rt22.1-py3.9,0b848dd4-e681-5599-be41-b5f6fccc6471,base,retired,pytorch-onnx_rt22.2-py3.10


In [50]:
print(sofware_spec_uid)

336b29df-e0e1-5e7d-b6a5-f6ab722625b2


### Guardamos modelo en deployment space

In [38]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.1',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=model,
    meta_props=metadata,
    training_data=train_data,
    training_target=train_labels)

/opt/conda/envs/Python-RT23.1-Premium/lib/python3.10/site-packages/charset_normalizer/api.py:105: UserWarning: Trying to detect encoding from a tiny portion of (22) byte(s).
  warn('Trying to detect encoding from a tiny portion of ({}) byte(s).'.format(length))


In [40]:
import json

published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "label_column": "l1",
    "schemas": {
      "input": [
        {
          "fields": [
            {
              "name": "f0",
              "type": "float"
            },
            {
              "name": "f1",
              "type": "float"
            },
            {
              "name": "f2",
              "type": "float"
            },
            {
              "name": "f3",
              "type": "float"
            },
            {
              "name": "f4",
              "type": "float"
            },
            {
              "name": "f5",
              "type": "float"
            },
            {
              "name": "f6",
              "type": "float"
            },
            {
              "name": "f7",
              "type": "float"
            },
            {
              "name": "f8",
              "type": "float"
            },
            {
              "name": "f9",
              "type": "fl

In [21]:
models_details = client.repository.list_models()


------------------------------------  ------------  ------------------------  ----------------  ----------  ----------------
ID                                    NAME          CREATED                   TYPE              SPEC_STATE  SPEC_REPLACEMENT
b6b85d11-85c5-4f41-98f3-bf3ff8350687  Scikit model  2025-01-27T19:56:17.002Z  scikit-learn_1.1  supported
------------------------------------  ------------  ------------------------  ----------------  ----------  ----------------


## 3. Despliegue modelo

In [42]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of scikit model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'b6b85d11-85c5-4f41-98f3-bf3ff8350687' started

#######################################################################################


initializing
Note: Software specification runtime-22.2-py3.10 is deprecated. Use supported software specification and corresponding model type instead when saving a model and deploy. For details, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html.
.............
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='9ea07ab9-14ff-4bcb-af16-45d00c1b9530'
------------------------------------------------------------------------------------------------




In [43]:
deployment_uid = client.deployments.get_id(created_deployment)

In [24]:
client.deployments.list()

------------------------------------  --------------------------  -----  ------------------------  -------------  ----------  ----------------
GUID                                  NAME                        STATE  CREATED                   ARTIFACT_TYPE  SPEC_STATE  SPEC_REPLACEMENT
9ea07ab9-14ff-4bcb-af16-45d00c1b9530  Deployment of scikit model  ready  2025-01-27T19:59:30.614Z  model          supported
------------------------------------  --------------------------  -----  ------------------------  -------------  ----------  ----------------


,GUID,NAME,STATE,CREATED,ARTIFACT_TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,9ea07ab9-14ff-4bcb-af16-45d00c1b9530,Deployment of scikit model,ready,2025-01-27T19:59:30.614Z,model,supported,


In [26]:
created_deployment=client.deployments.get_details(deployment_uid='9ea07ab9-14ff-4bcb-af16-45d00c1b9530')

Note: Software specification runtime-22.2-py3.10 is deprecated. Use supported software specification and corresponding model type instead when saving a model and deploy. For details, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html.


In [27]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://cpd-cpd.apps.cp4d-education-install.cp.fyre.ibm.com/ml/v4/deployments/9ea07ab9-14ff-4bcb-af16-45d00c1b9530/predictions


In [28]:
score_0 = list(score_data[0])
score_1 = list(score_data[1])

In [29]:
scoring_payload = {"input_data": [{"values": [score_0, score_1]}]}

In [34]:
predictions = client.deployments.score('9ea07ab9-14ff-4bcb-af16-45d00c1b9530', scoring_payload)

In [55]:
print(json.dumps(predictions, indent=2))


{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          5
        ],
        [
          4
        ]
      ]
    }
  ]
}
